In [ ]:
using Images, DataFrames, Colors,  HDF5, DecisionTree, Mocha

ENV["MOCHA_USE_NATIVE_EXT"] = "true"
using Mocha
backend = CPUBackend()
init(backend)


wd="/media/u01/analytics/scoring/k/USNS"
ssub = readtable("$(wd)/sample_submission.csv")
tm = readtable("$(wd)/train_masks.csv")

#Load Train
flist = readdir("$(wd)/train/")
numfiles=length(flist)
train = zeros(numfiles, 243600+3)
row=0
for f in flist
    #println(f)
    subject=float(split(replace(f,".tif",""),"_")[1])
    inum=float(split(replace(f,".tif",""),"_")[2])
    isMask=contains(f,"_mask") ? 1.0 : 0.0
    row=row+1
    fp="$(wd)/train/$(f)"
    if isfile(fp)
        img = imread(fp)
        train[row, 4:end] = reshape(img, 1, 243600)
        train[row,1]=subject
        train[row,2]=inum
        train[row,3]=isMask
        #println(f," ~ ", typeof(img)," ~ ",length(img))
        #println(f,"~",subject,"~",inum,"~",isMask)
    end
end


#http://stackoverflow.com/questions/30926050/how-to-save-an-nxnxn-array-or-matrix-into-a-file-in-julia-or-python
Pkg.add("HDF5")
using HDF5, JLD
r = rand(3, 3, 3)
save("data.jld", "data", r)
load("data.jld")["data"]


# ---- Mocha -----
img_width, img_height, img_channels = (256, 256, 3)
crop_size = (227, 227)
batch_size = 1  # could be larger if you want to classify a bunch of images at a time

layers = [ MemoryDataLayer(name="data", 
                           tops=[:data], 
                           batch_size=batch_size,
                           transformers=[(:data, DataTransformers.Scale(scale=255)),
                                         (:data, DataTransformers.SubMean(mean_file="model/ilsvrc12_mean.hdf5"))
                                        ],
                           data = Array[zeros(img_width, img_height, img_channels, batch_size)]
                         )
           CropLayer(name="crop", 
                     tops=[:cropped], 
                     bottoms=[:data], 
                     crop_size=crop_size
                    )
           ConvolutionLayer(name="conv1", 
                            tops=[:conv1], bottoms=[:cropped],
                            kernel=(11,11), 
                            stride=(4,4), 
                            n_filter=96, 
                            neuron=Neurons.ReLU()
                           )
           PoolingLayer(name="pool1", 
                        tops=[:pool1], 
                        bottoms=[:conv1],
                        kernel=(3,3), 
                        stride=(2,2), 
                        pooling=Pooling.Max()
                       )
           LRNLayer(name="norm1", 
                    tops=[:norm1], 
                    bottoms=[:pool1],
                    kernel=5, 
                    scale=0.0001, 
                    power=0.75
                   )
           ConvolutionLayer(name="conv2", 
                            tops=[:conv2], 
                            bottoms=[:norm1],
                            kernel=(5,5), 
                            pad=(2,2), 
                            n_filter=256, 
                            n_group=2, 
                            neuron=Neurons.ReLU()
                           )
            PoolingLayer(name="pool2", 
                         tops=[:pool2], 
                         bottoms=[:conv2],
                         kernel=(3,3), 
                         stride=(2,2), 
                         pooling=Pooling.Max()
                        )
             LRNLayer(name="norm2", 
                      tops=[:norm2], 
                      bottoms=[:pool2],
                      kernel=5, 
                      scale=0.0001, 
                      power=0.75)
             ConvolutionLayer(name="conv3", 
                              tops=[:conv3], 
                              bottoms=[:norm2],
                              kernel=(3,3), 
                              pad=(1,1), 
                              n_filter=384, 
                              neuron=Neurons.ReLU()
                             )
             ConvolutionLayer(name="conv4", 
                              tops=[:conv4], 
                              bottoms=[:conv3],
                              kernel=(3,3), 
                              pad=(1,1), 
                              n_filter=384, 
                              n_group=2, 
                              neuron=Neurons.ReLU())
             ConvolutionLayer(name="conv5", 
                              tops=[:conv5], 
                              bottoms=[:conv4],
                              kernel=(3,3), 
                              pad=(1,1), 
                              n_filter=256, 
                              n_group=2, 
                              neuron=Neurons.ReLU()
                             )
             PoolingLayer(name="pool5", 
                          tops=[:pool5], 
                          bottoms=[:conv5],
                          kernel=(3,3), 
                          stride=(2,2), 
                          pooling=Pooling.Max()
                         )
             InnerProductLayer(name="fc6", 
                               tops=[:fc6], 
                               bottoms=[:pool5],
                               output_dim=4096, 
                               neuron=Neurons.ReLU()
                             )
             InnerProductLayer(name="fc7", 
                               tops=[:fc7], 
                               bottoms=[:fc6],
                               output_dim=4096, 
                               neuron=Neurons.ReLU())
             InnerProductLayer(name="fc8", 
                               tops=[:fc8], 
                               bottoms=[:fc7],
                               output_dim=1000
                              )
             SoftmaxLayer(name="prob", 
                          tops=[:prob], 
                          bottoms=[:fc8]
                         )
]

net = Net("imagenet", backend, layers)
println(net)


